# M15, K=2.5 が妙にスコア良いから調べる
あと H12, K=0.5未満も

In [1]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_finance
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.endpoints.pricing import PricingInfo
import datetime
from statistics import mean
from statistics import median
import calendar
import datetime
from math import floor
import schedule
import time
import threading
import json

In [25]:
# OANDA API v20の口座IDとAPIトークン
accountID = "101-009-12609641-001"
access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")

In [3]:
# APIから取得したレートをPandasのDataFrameへ
def to_dataframe(r):
    data = []
    for raw in r.response['candles']:
        data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])

    # リストからPandas DataFrameへ変換
    df = pd.DataFrame(data)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df['Time'] = pd.to_datetime(df['Time'])
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].apply(float)
    return df

In [4]:
def get_data(count, gran, year, month, day, hour, minute, second):
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    _from = datetime.datetime(year=year, month=month, day=day,
                              hour=hour, minute=minute, second=second).strftime(fmt)
    params = {
        "count": count,
        "granularity": gran,
        'from': _from,
#         'dailyAlignment': 0
    }
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    df = to_dataframe(r)
    return df

In [5]:
# 第何何曜日（第nX曜日）の日付を返す関数
def get_day_of_nth_dow(year, month, nth, dow):
    '''dow: Monday(0) - Sunday(6)'''
    if nth < 1 or dow < 0 or dow > 6:
        return None

    first_dow, n = calendar.monthrange(year, month)
    day = 7 * (nth - 1) + (dow - first_dow) % 7 + 1

    return day if day <= n else None

In [9]:
def get_data_super(start, end, gran):
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12789646-001"
    access_token = "1ab53171ce4413f408de6da28ea23089-6426c3d3cc947dba212b3766d1be45c6"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    # xxxx-xx-xx～xxxx-xx-xx の期間のデータを取得してデータフレームを返す
    s_year, s_month, s_day, s_hour, s_minute, s_second = map(int, start.split('-'))
    e_year, e_month, e_day, e_hour, e_minute, e_second = map(int, end.split('-'))
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    from_ = datetime.datetime(year=s_year, month=s_month, day=s_day,
                              hour=s_hour, minute=s_minute, second=s_second).strftime(fmt)
    to_ = datetime.datetime(year=e_year, month=e_month, day=e_day,
                            hour=e_hour, minute=e_minute, second=e_second).strftime(fmt)
    to_dt = '{}-{}-{} {}:{}:{}'.format(e_year, e_month, e_day, e_hour, e_minute, e_second)
    to_unix = datetime.datetime.strptime(to_dt, '%Y-%m-%d %H:%M:%S').timestamp()
    df = pd.DataFrame()
    # 5000個制限に引っかからなければこっちの処理
    try:
        params = {
            "granularity": gran,
            'from': from_,
            'to': to_
        }
        r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
        api.request(r)
        df = to_dataframe(r)
    # 引っかかればこっちの処理
    except:
        i = True
        while i:
            df_tmp = get_data(count=5000, gran=gran, year=s_year, month=s_month, day=s_day,
                             hour=s_hour, minute=s_minute, second=s_second)
            # ケツが同じなら止める (最新迄用)
            if df.tail(1).reset_index(drop=True).equals(df_tmp.tail(1).reset_index(drop=True)):
                i = False
            else:
                df = pd.concat([df, df_tmp], ignore_index=True)
                index = df_tmp.tail(1).index[0]
                s_year = df_tmp.iat[index, 0].year
                s_month = df_tmp.iat[index, 0].month
                s_day = df_tmp.iat[index, 0].day
                s_hour = df_tmp.iat[index, 0].hour
                s_minute = df_tmp.iat[index, 0].minute
                s_second = df_tmp.iat[index, 0].second
                # print('{}-{}-{}-{}-{}-{}'.format(s_year, s_month, s_day, s_hour, s_minute, s_second))
                # 5000個ずつ取得してエンドの unix 超えたら止める
                if to_unix < df.iat[df.tail(1).index[0], 0].timestamp():
                    # はみ出した分をここで切り落とす
                    for i in range(len(df.index) - 5000, len(df.index)):
                        if to_unix == df.iat[i, 0].timestamp():
                            df = df[:i]
                            break
                    i = False
    return df

In [18]:
def set_profit(tmp_i, i, pos, entry_rate, exit_rate, assets, df, pos_df, pos_count):
    lot = 1
    fee = 0.008
    pos_count+=1
    if pos == 'entry_long':
        pips = (exit_rate - entry_rate - fee) * 100
        l_s = 'long'
    else:
        pips = (entry_rate - exit_rate - fee) * 100
        l_s = 'short'
    profit = lot * 25 * pips / 100
    assets = round(assets + profit, 4)
    tmp_se = pd.Series( [lot, l_s, df.iat[tmp_i,0], entry_rate, df.iat[i+1,0],
                        exit_rate, pips, profit, assets ], index=pos_df.columns )
    pos_df = pos_df.append( tmp_se, ignore_index=True )
    pos = 'None'
    return assets, pos_df, pos_count, pos

In [23]:
# バックテスト
def back_test(K, start, end, gran, ref, seed_money, kly_rate):
    range_ = 0
    range_list = []
    ave_range = 0
    least_5 = 0
    pos = 'None'
    entry_rate = 0
    pos_count = 0
    assets = seed_money
    lot = 0
    tmp_i = 0
    
    df = get_data_super(start, end, gran)
    pos_df = pd.DataFrame(columns=['lot', 'l_or_s', 'entry_date', 'entry_rate', 'exit_date', 'exit_rate', 'pips', 'profit', 'assets'])
    
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12609641-001"
    access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")

    # 実行
    for i in range(len(df.index) - 1):
        rate = df.iat[i, 5] # 当日の終値
        next_rate = df.iat[i+1, 2] # 次の足の始値（entry exit 価格）
        range_ = abs(round(df.iat[i, 3] - df.iat[i, 4], 3)) # |High - Low| レンジ幅
        range_list.append(range_)
    
        if i > ref:
            least_m = round(mean(range_list[i-ref-1:i-1]) * K, 3) # 直近5個のレンジ幅の平均
            red_range = round(df.iat[i, 3] - df.iat[i, 2], 3) # High - Open 赤幅(上げ幅)
            blue_range = round(df.iat[i, 2] - df.iat[i, 4], 3) # Open - Low 青幅(下げ幅)

            # 上昇トレンドと判断しロング
            if df.iat[i, 2] < df.iat[i, 5] and least_m < red_range: # 始値より終値の方が高く、5個平均よりも赤幅が大きいとき
                if pos == 'entry_long':
                    continue
                else:
                    # もし逆のポジションを持ってたら利確
                    if pos == 'entry_short':
                        exit_rate = next_rate
                        assets, pos_df, pos_count, pos = set_profit(tmp_i, i, pos, entry_rate, exit_rate,
                                                               assets, df, pos_df, pos_count)
                        if assets < 0:
                            break
                    
                    entry_rate = next_rate
                    pos = 'entry_long'
                    tmp_i = i+1
        
            # 下降トレンドと判断しショート
            if df.iat[i, 5] < df.iat[i, 2] and least_m < blue_range: # 終値より始値の方が高く、5個平均よりも青幅が大きいとき
                if pos == 'entry_short':
                    continue
                else:
                    # もし逆のポジションを持ってたら利確
                    if pos == 'entry_long':
                        exit_rate = next_rate
                        assets, pos_df, pos_count, pos = set_profit(tmp_i, i, pos, entry_rate, exit_rate,
                                                               assets, df, pos_df, pos_count)
                        if assets < 0:
                            break
        
                    entry_rate = next_rate
                    pos = 'entry_short'
                    tmp_i = i+1
                    
#     print('posision count: {}, seed_money: {}万円, assets: {}万円'.format(pos_count, seed_money/10000, round(assets/10000, 4)))
#     return pos_df, df, round(assets/10000, 4)
    return pos_df, df

In [10]:
df = get_data_super('2010-01-01-00-00-00', '2018-12-31-00-00-00', 'M15')

In [12]:
pos_df, df = back_test(K=0.5, start='2010-01-01-00-00-00', end='2018-12-31-00-00-00',
                                      gran='H12', ref=5, seed_money=10000000, kly_rate=0.05)

posision count: 1978, seed_money: 1000.0万円, assets: 1002.6011万円


In [26]:
# 2010年から2018年の9年間で検証
foot_gran_k = pd.DataFrame(columns=['K', 'gran', 'pos_count', 'pips'])
for i in range(5, 25+1):
    K = round(i * 0.1, 2)
    for gran in ['M5', 'M10', 'M15', 'M30',
                 'H1', 'H2', 'H3', 'H4', 'H6', 'H8', 'H12',
                 'D']:
        pos_df, df = back_test(K=K, start='2010-01-01-00-00-00', end='2018-12-31-00-00-00',
                               gran=gran, ref=5, seed_money=1000000, kly_rate=0.5)
        print('K: {}, gran: {}, pos_count: {}, pips: {:.4f}'.format(K, gran, pos_df.shape[0], pos_df['pips'].sum()))
        tmp_se = pd.Series([K, gran, pos_df.shape[0], round(pos_df['pips'].sum(), 4)], index=foot_gran_k.columns)
        foot_gran_k = foot_gran_k.append(tmp_se, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
foot_gran_k.sort_values(by='pips', ascending=False).head(10)

In [ ]:
# M15 検証
k_M15 = pd.DataFrame(columns=['K', 'pos_count', 'pips'])
for i in range(25, 50+1):
    K = round(i * 0.1, 2)
    pos_df, df = back_test(K=K, start='2010-01-01-00-00-00', end='2018-12-31-00-00-00',
                           gran='M15', ref=5, seed_money=1000000, kly_rate=0.5)
    print('K: {}, pos_count: {}, pips: {:.2f}'.format(K, pos_df.shape[0], pos_df['pips'].sum()))
    tmp_se = pd.Series([K, gran, pos_df.shape[0], round(pos_df['pips'].sum(), 2)], index=k_M15.columns)
    k_M15 = k_M15.append(tmp_se, ignore_index=True)

In [ ]:
# H12 検証
k_H12 = pd.DataFrame(columns=['K', 'pos_count', 'pips'])
for i in range(1, 5+1):
    K = round(i * 0.1, 2)
    pos_df, df = back_test(K=K, start='2010-01-01-00-00-00', end='2018-12-31-00-00-00',
                           gran='H12', ref=5, seed_money=1000000, kly_rate=0.5)
    print('K: {}, pos_count: {}, pips: {:.2f}'.format(K, pos_df.shape[0], pos_df['pips'].sum()))
    tmp_se = pd.Series([K, gran, pos_df.shape[0], round(pos_df['pips'].sum(), 2)], index=k_H12.columns)
    k_H12 = k_H12.append(tmp_se, ignore_index=True)